In [1]:
import pandas as pd
from h2o import h2o

In [2]:
pip install h2o

  Stored in directory: C:\Users\Sachin\AppData\Local\pip\Cache\wheels\eb\1a\72\252857fc093eaad70a7f91ddbcabf1dafd38fb5e6c02bcf9b3
  Stored in directory: C:\Users\Sachin\AppData\Local\pip\Cache\wheels\e1\41\5e\e201f95d90fc84f93aa629b6638adacda680fe63aac47174ab
Successfully built h2o tabulate
Note: you may need to restart the kernel to use updated packages.


In [43]:
import pandas as pd
from h2o import h2o

In [44]:

# Removing existing data from H2O Cluster

h2o.init(ip="localhost", port=54321)
h2o.remove_all()

Checking whether there is an H2O instance running at http://localhost:54321 . connected.


H2O cluster uptime:,17 mins 05 secs
H2O cluster timezone:,Asia/Kolkata
H2O data parsing timezone:,UTC
H2O cluster version:,3.26.0.6
H2O cluster version age:,8 days
H2O cluster name:,H2O_from_python_Sachin_l9r0z2
H2O cluster total nodes:,1
H2O cluster free memory:,762 Mb
H2O cluster total cores:,4
H2O cluster allowed cores:,4
H2O cluster status:,"locked, healthy"


In [45]:
# Loading HR Analytics Data from CSV File
full_data_frame = h2o.H2OFrame(pd.read_csv("dataset/HR_comma_sep.csv", index_col=None, header=0))

Parse progress: |█████████████████████████████████████████████████████████| 100%


In [46]:
full_data_frame

satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,left,promotion_last_5years,department,salary
0.38,0.53,2,157,3,0,1,0,sales,low
0.8,0.86,5,262,6,0,1,0,sales,medium
0.11,0.88,7,272,4,0,1,0,sales,medium
0.72,0.87,5,223,5,0,1,0,sales,low
0.37,0.52,2,159,3,0,1,0,sales,low
0.41,0.5,2,153,3,0,1,0,sales,low
0.1,0.77,6,247,4,0,1,0,sales,low
0.92,0.85,5,259,5,0,1,0,sales,low
0.89,1,5,224,5,0,1,0,sales,low
0.42,0.53,2,142,3,0,1,0,sales,low


In [47]:
# Defining categorical features
feature_columns = [
    'left',
    'Work_accident',
    'promotion_last_5years',
    'department'
]

# Defining continuous features
continuous_feature_columns = [
    'satisfaction_level',
    'last_evaluation',
    'number_project',
    'average_montly_hours',
    'time_spend_company',
    'salary'
]

In [48]:
feature_columns

['left', 'Work_accident', 'promotion_last_5years', 'department']

In [49]:
training_data_frame, test_data_frame = full_data_frame.split_frame(ratios=[.8])
training_data_frame[feature_columns] = training_data_frame[feature_columns].asfactor()
test_data_frame[feature_columns] = test_data_frame[feature_columns].asfactor()

In [50]:
print(training_data_frame[0, :])
print(test_data_frame[0, :])

satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,left,promotion_last_5years,department,salary
0.38,0.53,2,157,3,0,1,0,sales,low


satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,left,promotion_last_5years,department,salary
0.46,0.57,2,139,3,0,1,0,sales,low


In [51]:
feature_columns.extend(continuous_feature_columns)
training_data_frame = training_data_frame[feature_columns]
test_data_frame = test_data_frame[feature_columns]

In [52]:
feature_columns

['left',
 'Work_accident',
 'promotion_last_5years',
 'department',
 'satisfaction_level',
 'last_evaluation',
 'number_project',
 'average_montly_hours',
 'time_spend_company',
 'salary']

In [53]:
training_data_frame

left,Work_accident,promotion_last_5years,department,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,salary
1,0,0,sales,0.38,0.53,2,157,3,low
1,0,0,sales,0.8,0.86,5,262,6,medium
1,0,0,sales,0.11,0.88,7,272,4,medium
1,0,0,sales,0.72,0.87,5,223,5,low
1,0,0,sales,0.37,0.52,2,159,3,low
1,0,0,sales,0.41,0.5,2,153,3,low
1,0,0,sales,0.1,0.77,6,247,4,low
1,0,0,sales,0.92,0.85,5,259,5,low
1,0,0,sales,0.89,1,5,224,5,low
1,0,0,sales,0.42,0.53,2,142,3,low


In [54]:
test_data_frame

left,Work_accident,promotion_last_5years,department,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,salary
1,0,0,sales,0.46,0.57,2,139,3,low
1,0,0,sales,0.82,0.87,4,239,5,low
1,0,0,accounting,0.41,0.46,2,128,3,low
1,0,0,hr,0.45,0.57,2,134,3,low
1,0,0,hr,0.4,0.51,2,145,3,low
1,0,0,hr,0.45,0.55,2,140,3,low
1,0,0,technical,0.38,0.46,2,137,3,low
1,0,0,technical,0.45,0.5,2,126,3,low
1,0,0,technical,0.11,0.89,6,306,4,low
1,0,0,technical,0.45,0.48,2,158,3,low


In [55]:
# Initialize Random Forest Estimator
random_forest_model = h2o.H2ORandomForestEstimator(
    model_id="HREmployeeChurnPredictionRandomForest",
    ntrees=10,
    max_depth=10,
    min_rows=4,
    nfolds=10,
    seed=12345
)

In [58]:
random_forest_model.train(y='left', training_frame=training_data_frame)
y_output = random_forest_model.predict(test_data_frame);
random_forest_model_performance = random_forest_model.model_performance(test_data=test_data_frame)
print(random_forest_model_performance)
output = random_forest_model.predict(test_data_frame)
print(output)

drf Model Build progress: |███████████████████████████████████████████████| 100%
drf prediction progress: |████████████████████████████████████████████████| 100%

ModelMetricsBinomial: drf
** Reported on test data. **

MSE: 0.015862953232349146
RMSE: 0.12594821647148938
LogLoss: 0.14800734243749092
Mean Per-Class Error: 0.025835789045828728
AUC: 0.9895475887117996
pr_auc: 0.5392229661967602
Gini: 0.9790951774235992

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.48706896007061007: 


,,0,1,Error,Rate
0,0,2258.0,13.0,0.0057,(13.0/2271.0)
1,1,35.0,641.0,0.0518,(35.0/676.0)
2,Total,2293.0,654.0,0.0163,(48.0/2947.0)



Maximum Metrics: Maximum metrics at their respective thresholds


,metric,threshold,value,idx
0,max f1,0.487069,0.963910,142.0
1,max f2,0.375000,0.960402,158.0
2,max f0point5,0.648608,0.975686,125.0
3,max accuracy,0.500000,0.983712,140.0
4,max precision,0.997959,0.997409,2.0
5,max recall,0.000000,1.000000,399.0
6,max specificity,1.000000,0.999560,0.0
7,max absolute_mcc,0.487069,0.953614,142.0
8,max min_per_class_accuracy,0.235135,0.968935,180.0
9,max mean_per_class_accuracy,0.375000,0.974164,158.0



Gains/Lift Table: Avg response rate: 22.94 %, avg score: 23.13 %


,,group,cumulative_data_fraction,lower_threshold,lift,cumulative_lift,response_rate,score,cumulative_response_rate,cumulative_score,capture_rate,cumulative_capture_rate,gain,cumulative_gain
0,,1,0.102477,1.000000,4.345032,4.345032,0.996689,1.000000,0.996689,1.000000,0.445266,0.445266,334.503213,334.503213
1,,2,0.149983,0.988440,4.328328,4.339741,0.992857,0.997856,0.995475,0.999321,0.205621,0.650888,332.832840,333.974136
2,,3,0.200543,0.850000,4.330209,4.337338,0.993289,0.937302,0.994924,0.983685,0.218935,0.869822,333.020928,333.733818
3,,4,0.301663,0.100000,1.111810,3.256115,0.255034,0.291265,0.746907,0.751580,0.112426,0.982249,11.181049,225.611517
4,,5,0.401086,0.009707,0.059515,2.463726,0.013652,0.043379,0.565144,0.576028,0.005917,0.988166,-94.048509,146.372611
5,,6,0.536817,0.000169,0.010899,1.843542,0.002500,0.001720,0.422882,0.430817,0.001479,0.989645,-98.910133,84.354218
6,,7,1.000000,0.000000,0.022356,1.000000,0.005128,0.000000,0.229386,0.231270,0.010355,1.000000,-97.764376,0.000000




drf prediction progress: |████████████████████████████████████████████████| 100%


predict,p0,p1
1,0.0203802,0.97962
1,0.00204082,0.997959
1,0.000380228,0.99962
1,0,1
1,0,1
1,0,1
1,0.00204082,0.997959
1,0,1
1,0,1
1,0,1


In [59]:
# Initialize Naive Bayes Estimator
naive_bayes_model = h2o.H2ONaiveBayesEstimator(
    model_id="HREmployeeChurnPredictionNaiveBayes",
    nfolds=10,
    seed=123456
)

In [60]:
naive_bayes_model.train(y='left', training_frame=training_data_frame)
naive_bayes_model_performance = naive_bayes_model.model_performance(test_data=test_data_frame)
print(naive_bayes_model_performance)
output = naive_bayes_model.predict(test_data_frame)
print(output)

naivebayes Model Build progress: |████████████████████████████████████████| 100%

ModelMetricsBinomial: naivebayes
** Reported on test data. **

MSE: 0.04588947098872391
RMSE: 0.21421827883895414
LogLoss: 0.17459170590390455
Mean Per-Class Error: 0.07088801690468194
AUC: 0.9705734642351856
pr_auc: 0.8546855713154694
Gini: 0.9411469284703713

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.37313907344248526: 


,,0,1,Error,Rate
0,0,2182.0,89.0,0.0392,(89.0/2271.0)
1,1,81.0,595.0,0.1198,(81.0/676.0)
2,Total,2263.0,684.0,0.0577,(170.0/2947.0)



Maximum Metrics: Maximum metrics at their respective thresholds


,metric,threshold,value,idx
0,max f1,0.373139,0.875000,176.0
1,max f2,0.206214,0.898393,231.0
2,max f0point5,0.941473,0.903756,43.0
3,max accuracy,0.373139,0.942314,176.0
4,max precision,0.999851,1.000000,0.0
5,max recall,0.000036,1.000000,399.0
6,max specificity,0.999851,1.000000,0.0
7,max absolute_mcc,0.373139,0.837532,176.0
8,max min_per_class_accuracy,0.196941,0.927515,237.0
9,max mean_per_class_accuracy,0.206214,0.929112,231.0



Gains/Lift Table: Avg response rate: 22.94 %, avg score: 22.41 %


,,group,cumulative_data_fraction,lower_threshold,lift,cumulative_lift,response_rate,score,cumulative_response_rate,cumulative_score,capture_rate,cumulative_capture_rate,gain,cumulative_gain
0,,1,0.010180,9.998839e-01,4.359467,4.359467,1.000000,0.999946,1.000000,0.999946,0.044379,0.044379,335.946746,335.946746
1,,2,0.020020,9.996242e-01,4.359467,4.359467,1.000000,0.999753,1.000000,0.999851,0.042899,0.087278,335.946746,335.946746
2,,3,0.030200,9.992165e-01,4.359467,4.359467,1.000000,0.999422,1.000000,0.999706,0.044379,0.131657,335.946746,335.946746
3,,4,0.040041,9.986641e-01,4.359467,4.359467,1.000000,0.998983,1.000000,0.999529,0.042899,0.174556,335.946746,335.946746
4,,5,0.050221,9.981707e-01,4.359467,4.359467,1.000000,0.998431,1.000000,0.999306,0.044379,0.218935,335.946746,335.946746
5,,6,0.100102,9.932634e-01,4.329811,4.344690,0.993197,0.995975,0.996610,0.997646,0.215976,0.434911,332.981121,334.468960
6,,7,0.149983,9.786365e-01,4.270499,4.320015,0.979592,0.988546,0.990950,0.994620,0.213018,0.647929,327.049873,332.001526
7,,8,0.200204,5.553853e-01,2.886674,3.960465,0.662162,0.776272,0.908475,0.939848,0.144970,0.792899,188.667440,296.046535
8,,9,0.299966,1.045457e-01,1.527296,3.151244,0.350340,0.295770,0.722851,0.725641,0.152367,0.945266,52.729642,215.124401
9,,10,0.400068,1.993664e-02,0.206890,2.414531,0.047458,0.048397,0.553859,0.556186,0.020710,0.965976,-79.311002,141.453117




naivebayes prediction progress: |█████████████████████████████████████████| 100%


predict,p0,p1
1,0.0132317,0.986768
1,0.456829,0.543171
1,0.000290314,0.99971
1,0.00906239,0.990938
1,0.00615284,0.993847
1,0.00968913,0.990311
1,0.00106668,0.998933
1,0.007561,0.992439
1,0.000509799,0.99949
1,0.00484798,0.995152


In [61]:
# Initialize Gradient Boosting Estimator
gradient_boosting_model = h2o.H2OGradientBoostingEstimator(
    model_id="HREmployeeChurnPredictionGradientBoosting",
    ntrees=10,
    max_depth=10,
    stopping_tolerance=0.01,
    stopping_rounds=2,
    score_each_iteration=True,
    seed=1234567
)

In [62]:
gradient_boosting_model.train(y='left', training_frame=training_data_frame)
gradient_boosting_model_performance = gradient_boosting_model.model_performance(test_data=test_data_frame)
print(gradient_boosting_model)
output = gradient_boosting_model.predict(test_data_frame)

gbm Model Build progress: |███████████████████████████████████████████████| 100%
Model Details
H2OGradientBoostingEstimator :  Gradient Boosting Machine
Model Key:  HREmployeeChurnPredictionGradientBoosting


Model Summary: 


,,number_of_trees,number_of_internal_trees,model_size_in_bytes,min_depth,max_depth,mean_depth,min_leaves,max_leaves,mean_leaves
0,,10.0,10.0,31426.0,10.0,10.0,10.0,63.0,134.0,107.4




ModelMetricsBinomial: gbm
** Reported on train data. **

MSE: 0.02568593252226361
RMSE: 0.16026831415555481
LogLoss: 0.1516822111403662
Mean Per-Class Error: 0.0030031669760839375
AUC: 0.9999618438888829
pr_auc: 0.999534094220941
Gini: 0.9999236877777657

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.3734759738513501: 


,,0,1,Error,Rate
0,0,9144.0,13.0,0.0014,(13.0/9157.0)
1,1,17.0,2878.0,0.0059,(17.0/2895.0)
2,Total,9161.0,2891.0,0.0025,(30.0/12052.0)



Maximum Metrics: Maximum metrics at their respective thresholds


...



Scoring History: 


,,timestamp,duration,number_of_trees,training_rmse,training_logloss,training_auc,training_pr_auc,training_lift,training_classification_error
0,,2019-10-10 23:03:55,0.016 sec,0.0,0.427210,0.551321,0.500000,0.000000,1.00000,0.759791
1,,2019-10-10 23:03:56,0.142 sec,1.0,0.383214,0.459864,0.999628,0.061390,4.16304,0.009044
2,,2019-10-10 23:03:56,0.254 sec,2.0,0.345585,0.394473,0.999723,0.998123,4.16304,0.007053
3,,2019-10-10 23:03:56,0.349 sec,3.0,0.312529,0.343337,0.999777,0.998281,4.16304,0.006555
4,,2019-10-10 23:03:56,0.431 sec,4.0,0.282872,0.301225,0.999860,0.998530,4.16304,0.005144
5,,2019-10-10 23:03:56,0.509 sec,5.0,0.256616,0.266242,0.999897,0.998644,4.16304,0.004481
6,,2019-10-10 23:03:56,0.603 sec,6.0,0.232942,0.236309,0.999926,0.998733,4.16304,0.004066
7,,2019-10-10 23:03:56,0.697 sec,7.0,0.211758,0.210616,0.999939,0.998771,4.16304,0.003651
8,,2019-10-10 23:03:56,0.806 sec,8.0,0.192670,0.188243,0.999951,0.995701,4.16304,0.002987
9,,2019-10-10 23:03:56,0.899 sec,9.0,0.175568,0.168744,0.999957,0.999518,4.16304,0.002655



Variable Importances: 


,variable,relative_importance,scaled_importance,percentage
0,satisfaction_level,6.004061e+03,1.000000e+00,6.117121e-01
1,average_montly_hours,1.592371e+03,2.652156e-01,1.622356e-01
2,time_spend_company,1.307173e+03,2.177148e-01,1.331788e-01
3,last_evaluation,7.385150e+02,1.230026e-01,7.524217e-02
4,number_project,1.592344e+02,2.652111e-02,1.622329e-02
5,department,1.164683e+01,1.939826e-03,1.186615e-03
6,salary,2.172966e+00,3.619160e-04,2.213884e-04
7,promotion_last_5years,4.700232e-10,7.828422e-14,4.788741e-14
8,Work_accident,0.000000e+00,0.000000e+00,0.000000e+00



gbm prediction progress: |████████████████████████████████████████████████| 100%


In [63]:
# Initialize Generalized Linear Estimator
generalized_linear_model = h2o.H2OGeneralizedLinearEstimator(
    model_id="HREmployeeChurnPredictionGeneralizedLinear",
    nfolds=10,
    max_iterations=20,
    family="binomial",
    seed=1234567
)

In [65]:
generalized_linear_model.train(y='left', training_frame=training_data_frame)
generalized_linear_model_performance = generalized_linear_model.model_performance(test_data=test_data_frame)
print(generalized_linear_model_performance)
output = generalized_linear_model.predict(test_data_frame)
print(output)

glm Model Build progress: |███████████████████████████████████████████████| 100%

ModelMetricsBinomialGLM: glm
** Reported on test data. **

MSE: 0.035899140793655124
RMSE: 0.18947068584257334
LogLoss: 0.14363390087507644
Null degrees of freedom: 2946
Residual degrees of freedom: 2650
Null deviance: 3176.0258662897945
Residual deviance: 846.5782117577005
AIC: 1440.5782117577005
AUC: 0.97632647557706
pr_auc: 0.9494206623677149
Gini: 0.95265295115412

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.5057927366108389: 


,,0,1,Error,Rate
0,0,2215.0,56.0,0.0247,(56.0/2271.0)
1,1,57.0,619.0,0.0843,(57.0/676.0)
2,Total,2272.0,675.0,0.0383,(113.0/2947.0)



Maximum Metrics: Maximum metrics at their respective thresholds


,metric,threshold,value,idx
0,max f1,0.505793,0.916358,174.0
1,max f2,0.405696,0.922017,195.0
2,max f0point5,0.652183,0.927057,143.0
3,max accuracy,0.505793,0.961656,174.0
4,max precision,0.999746,1.000000,0.0
5,max recall,0.000599,1.000000,398.0
6,max specificity,0.999746,1.000000,0.0
7,max absolute_mcc,0.505793,0.891485,174.0
8,max min_per_class_accuracy,0.260720,0.940828,228.0
9,max mean_per_class_accuracy,0.405696,0.948064,195.0



Gains/Lift Table: Avg response rate: 22.94 %, avg score: 23.21 %


,,group,cumulative_data_fraction,lower_threshold,lift,cumulative_lift,response_rate,score,cumulative_response_rate,cumulative_score,capture_rate,cumulative_capture_rate,gain,cumulative_gain
0,,1,0.010859,0.997700,4.359467,4.359467,1.000000,0.998742,1.000000,0.998742,0.047337,0.047337,335.946746,335.946746
1,,2,0.020360,0.991176,4.359467,4.359467,1.000000,0.995536,1.000000,0.997246,0.041420,0.088757,335.946746,335.946746
2,,3,0.030200,0.980082,4.359467,4.359467,1.000000,0.984655,1.000000,0.993143,0.042899,0.131657,335.946746,335.946746
3,,4,0.040380,0.964995,4.359467,4.359467,1.000000,0.972572,1.000000,0.987957,0.044379,0.176036,335.946746,335.946746
4,,5,0.050221,0.955751,4.359467,4.359467,1.000000,0.961196,1.000000,0.982714,0.042899,0.218935,335.946746,335.946746
5,,6,0.100102,0.908219,4.300155,4.329912,0.986395,0.931340,0.993220,0.957114,0.214497,0.433432,330.015497,332.991174
6,,7,0.149983,0.844466,4.092561,4.250974,0.938776,0.878526,0.975113,0.930977,0.204142,0.637574,309.256128,325.097392
7,,8,0.200204,0.678039,3.858718,4.152577,0.885135,0.774442,0.952542,0.891711,0.193787,0.831361,285.871782,315.257747
8,,9,0.299966,0.141675,1.230734,3.180833,0.282313,0.367923,0.729638,0.717510,0.122781,0.954142,23.073401,218.083315
9,,10,0.400068,0.054909,0.162556,2.425624,0.037288,0.090903,0.556404,0.560725,0.016272,0.970414,-83.744359,142.562396




glm prediction progress: |████████████████████████████████████████████████| 100%


predict,p0,p1
1,0.125334,0.874666
1,0.302195,0.697805
1,0.03202,0.96798
1,0.0400495,0.95995
1,0.134488,0.865512
1,0.0817335,0.918267
1,0.0469823,0.953018
1,0.0836829,0.916317
1,0.0287871,0.971213
1,0.0469245,0.953075
